# 文本预处理

## 读入文本

In [2]:
import collections
import re

def read_time_machine():
    # 打开文本对象文件，创建文件对象f，f可迭代
    with open('./data/dive/timemachine.txt', 'r') as f:
        lines = [re.sub('[^a-z]+', ' ', line.strip().lower()) for line in f]
        # .strip：去掉前缀、后缀的空白字符，即空格、制表、换行符
        # .lower：大写转换为小写
        # re.sub：替换由非小写英文字符构成的长度至少为1（即非空）的字符串
    return lines # 列表


lines = read_time_machine()
print('# sentences %d' % len(lines))

# sentences 3221


## 分词

In [3]:
def tokenize(sentences, token='word'):
    """
    将一个句子划分成若干个词（token），转换为一个词的序列
    @sentences：列表，每个元素是句子
    @token：标志，做哪一级别的分词
    返回：两个维度的列表，第一维度是sentences的句子，第二维度是句子分词之后的序列
    """
    if token == 'word':
        return [sentence.split(' ') for sentence in sentences]
    elif token == 'char':
        return [list(sentence) for sentence in sentences]
    else:
        print('ERROR: unkown token type '+token)

tokens = tokenize(lines)
tokens[0:2]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells', ''], ['']]

## 建立字典

In [4]:
class Vocab(object):
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        '''
        构建一个字典（vocabulary），将每个词映射到一个唯一的索引编号
        @tokens：二维列表
        @min_freq：阈值，语料库出现的最少字数
        @use_special_tokens：标志，构建字典是否需要特殊的token
        
        '''
        # 去重，统计词频
        counter = count_corpus(tokens)  # 字典：{词：词频}
        self.token_freqs = list(counter.items()) # 构造列表
        # 增，删
        self.idx_to_token = [] # 记录字典需要维护的token
        if use_special_tokens:
            # padding, begin of sentence, end of sentence, unknown
            # pad：在小批量中数据不等长时做填充
            # bos：标记句子的开始
            # eos：标记句子的结束
            # unk：表示未登录词
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            self.idx_to_token += ['<pad>', '<bos>', '<eos>', '<unk>']
        else:
            self.unk = 0
            self.idx_to_token += ['<unk>']
        self.idx_to_token += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in self.idx_to_token]
        self.token_to_idx = dict() # 词到索引的映射
        for idx, token in enumerate(self.idx_to_token): # 词的下标和词
            self.token_to_idx[token] = idx

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        '''
        定义Vocab这个类的索引
        @tokens：列表/元组/字符串
        '''
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
            # 如tokens在token_to_idx中，返回tokens的值；否则返回self.unk
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        '''
        给定索引，返回对应的词
        '''
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(sentences):
    '''
    统计词频
    @sentences：二维列表
    '''
    tokens = [tk for st in sentences for tk in st] # 展开sentences得到一维列表
    return collections.Counter(tokens)  # 返回字典，记录每个词的出现次数

In [5]:
vocab = Vocab(tokens) # 构造实例
print(list(vocab.token_to_idx.items())[0:10]) # 前十个键值对

[('<unk>', 0), ('the', 1), ('time', 2), ('machine', 3), ('by', 4), ('h', 5), ('g', 6), ('wells', 7), ('', 8), ('i', 9)]


## 将词转为索引

In [6]:
# 使用字典，可将原文本中的句子从单词序列转换为索引序列
for i in range(8, 10):
    print('words:', tokens[i]) # tokens[i]：第i行tokenize之后得到的单词序列
    print('indices:', vocab[tokens[i]]) 

words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him', '']
indices: [1, 2, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 8]
words: ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
indices: [21, 22, 23, 24, 25, 17, 26, 27, 28, 29, 30, 31]


## 用现有工具进行分词

In [7]:
text = "Mr. Chen doesn't agree with my suggestion."

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm') # 导入已有的language
doc = nlp(text) # 可当作pipeline
print([token.text for token in doc])

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']


In [8]:
from nltk.tokenize import word_tokenize
from nltk import data
data.path.append('.data/dive/nltk_data')
print(word_tokenize(text))

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']
